<a href="https://colab.research.google.com/github/abouslima/AI-Makerspace/blob/master/VertexAI/PyCaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

! pip install {USER_FLAG} --upgrade google-cloud-aiplatform
! pip install {USER_FLAG} --upgrade google-cloud-storage
! pip install {USER_FLAG} --upgrade pillow
! pip install {USER_FLAG} --upgrade numpy

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
     |████████████████████████████████| 106 kB 53.7 MB/s 
     |████████████████████████████████| 106 kB 52.1 MB/s 
     |████████████████████████████████| 105 kB 61.4 MB/s 
     |████████████████████████████████| 105 kB 41.8 MB/s 
     |████████████████████████████████| 105 kB 68.7 MB/s 
     |████████████████████████████████| 105 kB 52.6 MB/s 
     |████████████████████████████████| 105 kB 57.1 MB/s 
     |████████████████████████████████| 104 kB 53.3 MB/s 
     |████████████████████████████████| 104 kB 47.9 MB/s 
     |████████████████████████████████| 103 kB 8.4 MB/s 
     |████████████████████████████████| 103 kB 53.3 MB/s 
     |████████████████████████████████| 103 kB 53.3 MB/s 
     |████████████████████████████████| 103 kB 28.5 MB/s 
     |████████████████████████████████| 97 kB 3.3 MB/s 
     |████████████████████████████████| 97 kB 4.3 MB/s 
     |████████████████

     |████████████████████████████████| 106 kB 5.3 MB/s 
     |████████████████████████████████| 111 kB 35.3 MB/s 
     |████████████████████████████████| 93 kB 2.1 MB/s 
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 1.41.1
    Uninstalling google-cloud-storage-1.41.1:
      Successfully uninstalled google-cloud-storage-1.41.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.10.0 requires google-cloud-storage<2.0.0dev,>=1.32.0, but you have google-cloud-storage 2.1.0 which is incompatible.


     |████████████████████████████████| 4.3 MB 5.2 MB/s 
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
from datetime import datetime

PROJECT_ID = "vertex-ai-makerspace"  # @param {type:"string"}

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [2]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [3]:
BUCKET_NAME = "gs://vertex-ai-makerspace-pycaret-45454"  # @param {type:"string"}
REGION = "europe-west4"  # @param {type:"string"}

In [4]:
! gsutil mb -p $PROJECT_ID -l $REGION $BUCKET_NAME
! gsutil ls -al $BUCKET_NAME

Creating gs://vertex-ai-makerspace-pycaret-45454/...


In [5]:
from google.cloud import aiplatform
#from google.cloud.aiplatform import gapic as aip

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

In [6]:
TRAIN_VERSION = "scikit-learn-cpu.0-23"
DEPLOY_VERSION = "sklearn-cpu.0-23"

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

In [7]:
MACHINE_TYPE = "n1-standard"

VCPU = "8"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "8"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

Train machine type n1-standard-8
Deploy machine type n1-standard-8


In [8]:
JOB_NAME = "custom_job_" + TIMESTAMP

In [9]:
%%writefile task.py

from google.cloud import storage
import pandas as pd
from pycaret.classification import *
import os
import joblib

dataset = pd.read_csv("https://raw.githubusercontent.com/DigitalProductschool/AI-Makerspace/master/PyCaret-Classification/UniversalBank.csv")
dataset.columns = [i.replace(" ", "") for i in dataset.columns]
dataset.drop(["ID","ZIPCode"],axis=1,inplace=True)
cat_cols = ["Family","Education","SecuritiesAccount","CDAccount","Online","CreditCard"]
data = dataset.sample(frac=0.9, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)
exp_1 = setup(data = data, 
                    session_id=123, 
                    target = 'PersonalLoan', 
                    categorical_features=cat_cols,
                    normalize=True, 
                    normalize_method='minmax',
                    transformation=True,
                    use_gpu=False,
                    log_experiment=True,
                    experiment_name='loan1',
                    silent=True)

best = compare_models()

tuned_model = tune_model(best, optimize = 'AUC') # Optimize - Measure used to select the best model through hyperparameter tuning.
final_model = finalize_model(tuned_model, model_only=True)


artifact_filename = 'model.joblib'

# Save model artifact to local filesystem (doesn't persist)
local_path = artifact_filename
joblib.dump(final_model, local_path)

# Upload model artifact to Cloud Storage
model_directory = os.environ['AIP_MODEL_DIR']
storage_path = os.path.join(model_directory, artifact_filename)
blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())
blob.upload_from_filename(local_path)

Writing task.py


In [10]:
job = aiplatform.CustomTrainingJob(
    display_name=JOB_NAME,
    script_path="task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["pycaret", "numpy==1.19.5"],
    model_serving_container_image_uri=DEPLOY_IMAGE
)

MODEL_DISPLAY_NAME = "pycaret-" + TIMESTAMP

model = job.run(
    model_display_name=MODEL_DISPLAY_NAME,
    replica_count=1,
    machine_type=TRAIN_COMPUTE,
)

INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://vertex-ai-makerspace-pycaret-45454/aiplatform-2022-02-22-13:56:20.457-aiplatform_custom_trainer_script-0.1.tar.gz.
INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://vertex-ai-makerspace-pycaret-45454/aiplatform-custom-training-2022-02-22-13:56:20.654 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/4100468087092936704?project=249560904503
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/8331529568263340032?project=249560904503
INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/249560904503/locations/europe-west4/trainingPipelines/4100468087092936704 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/249560904503/lo

In [11]:
DEPLOYED_NAME = "pycaret-" + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1


endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/249560904503/locations/europe-west4/endpoints/4612020269962231808/operations/7149661221032034304
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/249560904503/locations/europe-west4/endpoints/4612020269962231808
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/249560904503/locations/europe-west4/endpoints/4612020269962231808')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/249560904503/locations/europe-west4/endpoints/4612020269962231808
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/249560904503/locations/europe-west4/endpoints/4612020269962231808/operations/7336560605567909888
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/249560

In [149]:
endpoint_name="projects/249560904503/locations/europe-west4/endpoints/3561837131854774272"
endpoint = aiplatform.Endpoint(endpoint_name=endpoint_name)

In [13]:
import pandas as pd
dataset = pd.read_csv("https://raw.githubusercontent.com/DigitalProductschool/AI-Makerspace/master/PyCaret-Classification/UniversalBank.csv")
dataset.columns = [i.replace(" ", "") for i in dataset.columns]
dataset.drop(["ID","ZIPCode"],axis=1,inplace=True)
cat_cols = ["Family","Education"]
data = dataset.sample(frac=0.9, random_state=786)
data_unseen = dataset.drop(data.index)
data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

In [14]:
X_train = pd.get_dummies(data, columns=cat_cols).drop(["PersonalLoan"], axis=1)
X_test = pd.get_dummies(data_unseen, columns=cat_cols).drop(["PersonalLoan"], axis=1)

In [19]:
from sklearn.preprocessing import StandardScaler

In [20]:
sc = StandardScaler()

In [22]:
X_train = sc.fit_transform(X_train)

In [23]:
X_test = sc.transform(X_test)

In [231]:
len(X_test.values.tolist())

500

In [24]:
predictions = endpoint.predict(instances=X_test.tolist())

In [265]:
print(predictions)

Prediction(predictions=[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(data_unseen.PersonalLoan.values, predictions[0])

0.856